In [1]:
import pandas as pd
import numpy as np
import glob
from tkinter.filedialog import askopenfilename, askopenfilenames, Tk
import json
import os

In [3]:
initial_dir = 'g:\\BowlNeuron\\BOF\\TimeStamps\\'

def select_csv_files(initial_dir):
    root = Tk()    
    #root.wm_attributes('-topmost', 1)
    root.withdraw()
    
    files = askopenfilenames(initialdir = initial_dir, title="Выберите CSV файлы", filetypes=[("CSV Files", "*.csv")])
    #root.lift()
    return files

def merge_csv(files):
    
    # Список для хранения загруженных данных из CSV файлов
    dataframes = []
    
    # Список для хранения количества элементов в каждом файле
    num_elements = []
    
    # Чтение каждого CSV файла и добавление его в список
    for i, file in enumerate(files):
        df = pd.read_csv(file, header=None)
        
        # Считаем количество элементов и добавляем в список
        num_elements.append(len(df))
        
        # recalulation TS for inter-session delay 
        df_list = df[0].tolist() 
        first_ts = df_list[0]
        diff_array = np.diff(df_list)
        mean_diff = np.mean(diff_array)       
        
        print(first_ts)
        print(mean_diff)
        
        if i != 0:
            # Выполняем операции над списком
            df_list = [x - (first_ts - last_ts) + mean_diff for x in df_list]        

        last_ts = df_list[-1]
        print(last_ts)
        
        # Обратно преобразуем список в DataFrame
        df = pd.DataFrame(df_list)        
        dataframes.append(df)
        
    merged_df = pd.concat(dataframes)    
    return merged_df, num_elements
    

# Выбор CSV файлов пользователем
csv_files = select_csv_files(initial_dir)
csv_files = sorted(csv_files)
print(csv_files)

# Получение имени для сохранения файла
save_name = os.path.basename(csv_files[0])[:7]

# Сортировка и объединение CSV файлов
merged_data, num_elements = merge_csv(csv_files)

# Создание пути для сохранения файла
save_path = os.path.join(initial_dir, f"{save_name}.csv")
#save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])

# Сохранение результата в новый CSV файл
merged_data.to_csv(save_path, index=False, header=False)

# Создание JSON объекта
json_data = {'num_elements': num_elements}
print(num_elements)

# Запись JSON в файл
with open(os.path.join(initial_dir, f"{save_name}.json"), 'w') as f:
    json.dump(json_data, f)

['G:/BowlNeuron/BOF/TimeStamps/BOF_H02_1T_Mini_TS.csv', 'G:/BowlNeuron/BOF/TimeStamps/BOF_H02_2T_Mini_TS.csv', 'G:/BowlNeuron/BOF/TimeStamps/BOF_H02_3T_Mini_TS.csv', 'G:/BowlNeuron/BOF/TimeStamps/BOF_H02_4T_Mini_TS.csv', 'G:/BowlNeuron/BOF/TimeStamps/BOF_H02_5T_Mini_TS.csv']
638421433762938624
506202.89275460225
638421439757393280
638421445973276928
506128.9470573334
6.384214457519845e+17
638421458221916032
506132.69745841424
6.384214517466202e+17
638421470442022528
506132.94604407664
6.384214577412588e+17
638421482663296000
506129.94139998313
6.384214637358618e+17
[11843, 11844, 11844, 11844, 11844]


In [5]:
initial_dir = 'g:\\BowlNeuron\\BOF\\'
out_dir = 'g:\\BowlNeuron\\BOF\\'

def split_csv_by_json(csv_file, json_file):
    
    # Чтение JSON файла
    with open(json_file, 'r') as f:
        json_data = json.load(f)
    num_frames = json_data['num_elements']
    
    # Чтение CSV файла
    df = pd.read_csv(csv_file, header=None)       
    
    # Разделение CSV файла на несколько файлов
    #chunks = [df.iloc[i:i+num_frames] for i in range(0, len(df), num_frames)] 
    
    chunks = []
    
    # Разделение CSV файла на несколько файлов
    start_frame = 0
    for i in range(len(num_frames)):
        chunk = df.iloc[start_frame:start_frame+num_frames[i]]
        chunks.append(chunk)
        start_frame += num_frames[i]
    return chunks
    
csv_file = askopenfilename(initialdir = initial_dir, title="Выберите CSV файл", filetypes=[("CSV Files", "*.csv")])
json_file = askopenfilename(initialdir = initial_dir, title="Выберите json файл", filetypes=[("JSON Files", "*.json")])

chunks = split_csv_by_json(csv_file, json_file)

# Сохранение каждого куска в отдельный файл
save_name = os.path.basename(csv_file)

for i, chunk in enumerate(chunks):
    save_path = os.path.join(out_dir, f"{save_name[:-4]}_{i+1}T.csv")
    chunk.to_csv(save_path, index=False, header=False)